# Propuesta de visualizaciones:

1. [Evolución de duración media de viaje (2018–2025)][1]
2. Distribución de motivos de viaje
3. Mapa de calor: motivos de viaje por comunidad (rural vs ciudad)
4. Comparativa gasto medio turista "nuevo modelo de viaje" vs general (por ejemplo, nómadas digitales vs familias)
5. Línea temporal: turismo experiencial vs tradicional

[1] Ver si hay datos disponibles e 2025 o tan solo hasta 2024.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Evolución tipo de visitantes (excursionistas vs turistas) por CCAA

In [2]:
df_long = visitantes_ccaa_pivot.melt(
    id_vars=["AÑO", "CCAA_DESTINO"],
    value_vars=["Turista", "Excursionista"],
    var_name="TIPO_VISITANTE",
    value_name="VISITANTES"
)

NameError: name 'visitantes_ccaa_pivot' is not defined